<a href="https://colab.research.google.com/github/Hujaifa98/AIProjects/blob/main/Kaggle_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt

Now we have to import the given csv files


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
raw_train = pd.read_csv('train.csv')
raw_train['is_test'] = 0
raw_test = pd.read_csv('test.csv')
raw_test['is_test'] = 1
all_data = pd.concat((raw_train , raw_test) , axis=0)

In [ ]:
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,1
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,1


In [ ]:
all_data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
is_test           0
dtype: int64

In [ ]:
all_data['SibSp'].value_counts()

0    891
1    319
2     42
4     22
3     20
8      9
5      6
Name: SibSp, dtype: int64

In [ ]:
def split_name (name):
  full_name = name.str.split(', ', n=0, expand=True)
  last_name = full_name[0]
  titles = full_name[1].str.split('.', n=0, expand=True)
  titles = titles[0]
  return titles


In [ ]:
all_data['Name'] = split_name(all_data['Name'])

In [ ]:
def cabin_first_letter(cabin):
  cabin_first = cabin.str[0]
  return cabin_first

In [ ]:
all_data['Cabin'].fillna('Z' , inplace = True)
all_data['Cabin'] = cabin_first_letter(all_data['Cabin'])

In [ ]:
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test
0,1,0.0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,Z,S,0
1,2,1.0,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C,C,0
2,3,1.0,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,Z,S,0
3,4,1.0,1,Mrs,female,35.0,1,0,113803,53.1000,C,S,0
4,5,0.0,3,Mr,male,35.0,0,0,373450,8.0500,Z,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,Z,S,1
414,1306,NaN,1,Dona,female,39.0,0,0,PC 17758,108.9000,C,C,1
415,1307,NaN,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,Z,S,1
416,1308,NaN,3,Mr,male,NaN,0,0,359309,8.0500,Z,S,1


In [ ]:
all_data = pd.get_dummies(all_data, columns = ['Name' , 'Pclass' , 'Sex' , 'Embarked' , 'Cabin'])

In [ ]:
all_data

,PassengerId,Survived,Age,SibSp,Parch,Ticket,Fare,is_test,Name_Capt,Name_Col,Name_Don,Name_Dona,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_the Countess,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_Z
0,1,0.0,22.0,1,0,A/5 21171,7.2500,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
1,2,1.0,38.0,1,0,PC 17599,71.2833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2,3,1.0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1
3,4,1.0,35.0,1,0,113803,53.1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
4,5,0.0,35.0,0,0,373450,8.0500,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,NaN,0,0,A.5. 3236,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
414,1306,NaN,39.0,0,0,PC 17758,108.9000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
415,1307,NaN,38.5,0,0,SOTON/O.Q. 3101262,7.2500,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
416,1308,NaN,NaN,0,0,359309,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
for_age_train = all_data.drop(['Survived' , 'is_test' , 'Ticket' , 'Fare'] , axis=1)
for_age_train = for_age_train.dropna(axis=0)
x_age_train = for_age_train.drop(['Age'] , axis=1)
y_age_train = for_age_train['Age']

In [ ]:
i = Input(shape= x_age_train.shape[1])
x = Dense(64, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(i)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.20)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.20)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.20)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.20)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.20)(x)
x = Dense(1 , activation='linear')(x)
agemodel = Model(i , x)

In [ ]:
agemodel.compile(optimizer='rmsprop' , loss='mean_squared_error')

In [ ]:
agemodel.fit(x_age_train.to_numpy() , y_age_train.to_numpy() , epochs=600)

Epoch 1/600
33/33 [==============================] - 0s 3ms/step - loss: 530.5478
Epoch 2/600
33/33 [==============================] - 0s 2ms/step - loss: 458.1100
Epoch 3/600
33/33 [==============================] - 0s 2ms/step - loss: 437.7829
Epoch 4/600
33/33 [==============================] - 0s 2ms/step - loss: 428.0513
Epoch 5/600
33/33 [==============================] - 0s 3ms/step - loss: 409.0886
Epoch 6/600
33/33 [==============================] - 0s 3ms/step - loss: 383.7653
Epoch 7/600
33/33 [==============================] - 0s 2ms/step - loss: 333.9934
Epoch 8/600
33/33 [==============================] - 0s 3ms/step - loss: 309.0154
Epoch 9/600
33/33 [==============================] - 0s 2ms/step - loss: 330.7959
Epoch 10/600
33/33 [==============================] - 0s 2ms/step - loss: 297.8945
Epoch 11/600
33/33 [==============================] - 0s 3ms/step - loss: 318.7176
Epoch 12/600
33/33 [==============================] - 0s 3ms/step - loss: 252.4032
Epoch 13/600


In [ ]:
null_age_train = all_data.drop(['Survived' , 'is_test' , 'Ticket' , 'Fare'] , axis=1)
null_age_train = null_age_train.loc[null_age_train['Age'].isnull()]
null_age_train = null_age_train.drop(['Age'] ,axis=1)

In [ ]:
age_prediction = agemodel.predict(null_age_train.to_numpy())
age_prediction.shape

(263, 1)

In [ ]:
all_data.drop(['Ticket'] , axis=1 , inplace=True)
all_data.set_index('PassengerId' , inplace=True)

In [ ]:

all_data

,Survived,Age,SibSp,Parch,Fare,is_test,Name_Capt,Name_Col,Name_Don,Name_Dona,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_the Countess,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_Z
PassengerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,22.000000,1,0,7.2500,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
2,1.0,38.000000,1,0,71.2833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,1.0,26.000000,0,0,7.9250,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1
4,1.0,35.000000,1,0,53.1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
5,0.0,35.000000,0,0,8.0500,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,27.283699,0,0,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1
1306,NaN,39.000000,0,0,108.9000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
1307,NaN,38.500000,0,0,7.2500,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
train = all_data[all_data['is_test'] == 0]
test = all_data[all_data['is_test'] == 1]

In [ ]:
x_train = train.drop(['Survived' , 'is_test'] ,axis=1)
y_train = train['Survived']
x_test = test.drop(['Survived' , 'is_test'] ,axis=1)

In [ ]:
x_train.shape

(891, 39)

In [ ]:
i = Input(shape=x_train.shape[1])
x = Dense(64, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(i)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(128, activation='relu' , kernel_initializer='normal', bias_initializer='zeros')(x)
x = Dropout(.25)(x)
x = Dense(1,activation='sigmoid')(x)
model = Model(i , x)

In [ ]:
model.compile(optimizer='adam' , loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
r = model.fit(x_train.to_numpy() , y_train.to_numpy() , epochs=500 )

Epoch 1/500
28/28 [==============================] - 0s 6ms/step - loss: 0.6832 - accuracy: 0.5915
Epoch 2/500
28/28 [==============================] - 0s 5ms/step - loss: 0.6417 - accuracy: 0.6184
Epoch 3/500
28/28 [==============================] - 0s 5ms/step - loss: 0.6143 - accuracy: 0.6633
Epoch 4/500
28/28 [==============================] - 0s 5ms/step - loss: 0.5773 - accuracy: 0.6768
Epoch 5/500
28/28 [==============================] - 0s 5ms/step - loss: 0.5467 - accuracy: 0.7306
Epoch 6/500
28/28 [==============================] - 0s 5ms/step - loss: 0.5602 - accuracy: 0.7452
Epoch 7/500
28/28 [==============================] - 0s 6ms/step - loss: 0.5410 - accuracy: 0.7621
Epoch 8/500
28/28 [==============================] - 0s 5ms/step - loss: 0.5296 - accuracy: 0.7464
Epoch 9/500
28/28 [==============================] - 0s 5ms/step - loss: 0.4979 - accuracy: 0.7834
Epoch 10/500
28/28 [==============================] - 0s 5ms/step - loss: 0.5504 - accuracy: 0.7621
Epoch 11/

In [ ]:
prediction = model.predict(x_test.to_numpy())
prediction = np.round(prediction).flatten()
prediction.shape

(418,)

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = raw_test['PassengerId']
submission['Survived'] = prediction

In [ ]:
submission

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [ ]:
submission.to_csv('titanic_keras_cs.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:

!cp titanic_keras_cs.csv "drive/My Drive/"